In [1]:
import os
import numpy as np
from os.path import isfile, join
from pathlib import Path
import pandas as pd
#from ipynb.fs.full.BB_functions import files_from_folder, preprocess

In [2]:
input_path = Path(r'C:\Users\UCPH\gabi\thesis analysis\test1')
input_path2 = Path(r'C:\Users\UCPH\gabi\thesis analysis\test2')
save_path = Path(r'C:\Users\UCPH\gabi\thesis analysis')

combined_path = os.path.join(input_path, input_path2)

In [4]:
df = files_from_folder(combined_path)

In [5]:
df

1,Nose_x,FrontLimb_x,HindLimb_x,TailBase_x,Start_x,End_x,Nose_y,FrontLimb_y,HindLimb_y,TailBase_y,Start_y,End_y,Nose_likelihood,FrontLimb_likelihood,HindLimb_likelihood,TailBase_likelihood,Start_likelihood,End_likelihood,Name File
3,181.575607,142.569519,62.056583,55.628613,32.686756,1112.671997,227.590363,233.456985,226.868454,191.981308,233.465683,220.942612,1.000000,0.992205,0.999664,0.999892,0.996044,0.999849,"1_POST_BB_15_H0,2_KI_M_T1"
4,202.752014,145.496994,72.242058,72.860168,36.782040,1112.708740,225.885925,234.878067,221.357727,205.209290,230.039444,220.584717,0.999996,0.999848,0.989090,0.999715,0.999868,0.999867,"1_POST_BB_15_H0,2_KI_M_T1"
5,202.752655,145.553162,72.486900,72.640656,35.920845,1112.722290,225.888199,234.864944,221.085968,205.038559,229.206589,220.572021,0.999996,0.999859,0.989455,0.999701,0.999917,0.999866,"1_POST_BB_15_H0,2_KI_M_T1"
6,226.492004,189.416916,116.875633,91.615646,36.225803,1113.403198,225.756989,230.018616,236.289902,210.197876,230.437134,221.220551,0.999999,0.999641,0.999813,0.999920,0.999904,0.999834,"1_POST_BB_15_H0,2_KI_M_T1"
7,244.978149,195.232849,117.810814,111.228165,34.076157,1114.256470,224.095917,228.623718,233.531265,210.584732,229.465210,221.240082,0.999999,0.999948,0.999902,0.999949,0.997012,0.999675,"1_POST_BB_15_H0,2_KI_M_T1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,1203.319580,1167.914917,1144.543945,1111.087524,25.176291,1108.187500,238.343262,234.402145,234.387344,221.850952,227.374435,217.696838,0.999997,0.999796,0.989164,0.998271,0.999931,0.598016,"1_PRE_BB_37_H0,8_WT_M_T4"
87,1203.306641,1167.927124,1144.628784,1111.127197,25.314110,1108.174927,238.314209,234.348450,234.411850,221.887390,227.199097,217.902237,0.999997,0.999795,0.989757,0.998333,0.999912,0.559191,"1_PRE_BB_37_H0,8_WT_M_T4"
88,1202.397705,1167.772827,1147.985474,1116.746582,25.202160,1109.177856,234.186935,236.497589,239.568069,237.369385,227.330368,229.688782,0.999998,0.999871,0.949231,0.999340,0.999928,0.029175,"1_PRE_BB_37_H0,8_WT_M_T4"
89,1189.521851,1175.634277,1154.050903,1114.361084,25.198122,1115.891357,236.058151,235.648041,239.022568,231.595230,227.399902,231.952866,0.998705,0.999117,0.991880,0.999832,0.999925,0.041935,"1_PRE_BB_37_H0,8_WT_M_T4"


In [ ]:
# Split the 'Name File' column into multiple columns
df[['Cohort', 'Pre-Post', 'Test_Type', 'Animal_ID', 'Drug_Dose', 'Genotype', 'Gender', 'Trial']] = df['Name File'].str.split('_', expand=True)

# Drop the original 'Name File' column if needed
# df.drop('Name File', axis=1, inplace=True)

# Split 'drug_dose' into 'Drug' and 'Dose' columns
df[['Drug', 'Dose']] = df['Drug_Dose'].str.extract(r'([A-Za-z]+)(\d+,\d+)')

# Extract only numbers from the 'Trial' column
df['Trial'] = df['Trial'].str.extract('(\d+)')

# Print the updated DataFrame
df

In [ ]:
# Iterate through columns and set values to NaN based on condition
for col in df.columns:
    if col.endswith('_likelihood'):
        mask = df[col] < 0.7
        prefix = col[:-11]  # Remove '_likelihood' to get the prefix
        for suffix in ['x', 'y']:
            column_to_set_nan = f'{prefix}_{suffix}'
            df.loc[mask, column_to_set_nan] = np.nan

In [ ]:
# Function to calculate the adjusted mean
def calculate_adjusted_mean(group):
    filtered_start_x = group[group['Start_likelihood'] > 0.95]['Start_x']
    filtered_start_y = group[group['Start_likelihood'] > 0.95]['Start_y']
    filtered_end_x = group[group['End_likelihood'] > 0.95]['End_x']
    filtered_end_y = group[group['End_likelihood'] > 0.95]['End_y']
    
    start_x_adjusted_mean = filtered_start_x.mean()
    start_y_adjusted_mean = filtered_start_y.mean()
    end_x_adjusted_mean = filtered_end_x.mean()
    end_y_adjusted_mean = filtered_end_y.mean()
    
    group['Start_x_adjust'] = start_x_adjusted_mean
    group['Start_y_adjust'] = start_y_adjusted_mean
    group['End_x_adjust'] = end_x_adjusted_mean
    group['End_y_adjust'] = end_y_adjusted_mean
    
    return group

# Group by 'Name File' and apply the calculation
df = df.groupby('Name File').apply(calculate_adjusted_mean)

# Fill NaN values in the new columns with appropriate values
df['Start_x_adjust'].fillna(df['Start_x'], inplace=True)
df['Start_y_adjust'].fillna(df['Start_y'], inplace=True)
df['End_x_adjust'].fillna(df['End_x'], inplace=True)
df['End_y_adjust'].fillna(df['End_y'], inplace=True)

# Drop the original 'Start_likelihood' and 'End_likelihood' columns if needed
# df.drop(['Start_likelihood', 'End_likelihood'], axis=1, inplace=True)

df

In [ ]:
# Define a function to determine Framerate and y_pixels based on Cohort
def set_framerate_and_ypixels(row):
    if row['Cohort'] == 1:
        return 15, 720
    elif row['Cohort'] == 2:
        return 60, 1080
    else:
        return np.nan, np.nan
    
# Convert 'Cohort' to integer
df['Cohort'] = df['Cohort'].astype(int)

# Apply the function to create the 'Framerate' and 'y_pixels' columns
df[['Framerate', 'y_pixels']] = df.apply(set_framerate_and_ypixels, axis=1, result_type='expand')
df['Framerate'] = df['Framerate'].astype(int)

In [ ]:
#Get distance to end
df['Nose_to_end'] = 75 - (df['Nose_x']-df['Start_x_adjust'])/((df['End_x_adjust']-df['Start_x_adjust'])/75)
df['TailBase_to_end'] = 75 - (df['TailBase_x']-df['Start_x_adjust'])/((df['End_x_adjust']-df['Start_x_adjust'])/75)
df['Time_to_cross_s'] = df.groupby('Name File')['Framerate'].transform('size') / df['Framerate']

# Create 'Failed_to_end' column with True/False values based on if it's more than 5 cm away from end on the last frame of each trial
df['Failed_to_end'] = df.groupby('Name File')['TailBase_to_end'].transform(lambda x: x.iloc[-1] > 1 )

def calculate_weighted_position(row):
    if row['TailBase_to_end'] >= 75:
        return row['Start_y_adjust']
    elif row['TailBase_to_end'] <= 0:
        return row['End_y_adjust']
    else:
        weight_start = (row['TailBase_to_end'] / 75)
        weight_end = 1 - weight_start
        return (weight_start * row['Start_y_adjust']) + (weight_end * row['End_y_adjust'])
    
# Apply the function to create the 'Position_y' column
df['Position_y'] = df.apply(calculate_weighted_position, axis=1)

# Apply the function to create the 'Position_y' column
df['Position_y'] = df.apply(calculate_weighted_position, axis=1)
#Get distance traveled
df['Nose_diff'] = df['Nose_to_end'].diff().abs()
df['TailBase_diff'] = df['TailBase_to_end'].diff().abs()
df['Distance_traveled'] = (df['Nose_diff'] + df['TailBase_diff']) / 2
df['Absolute_distance_traveled'] = df.groupby('Name File')['Distance_traveled'].transform('sum')

#Determine speed
df['Speed_cm/s'] = df['Distance_traveled']*df['Framerate']

# Calculate the condition for 'Start_box_nose' and set to False when the condition is not met
threshold = df['Start_x_adjust'] + (10 * ((df['End_x_adjust'] - df['Start_x_adjust']) / 75))
df['Start_box_nose'] = (df['Nose_x'] - df['Start_x_adjust']) < threshold
df['Start_box_nose'] = df['Start_box_nose'].fillna(False)

# Calculate the condition for 'Start_box_tailbase' and set to False when the condition is not met
df['Start_box_tailbase'] = (df['TailBase_x'] - df['Start_x_adjust']) < threshold
df['Start_box_tailbase'] = df['Start_box_tailbase'].fillna(False)

#Pawslips
df['FrontLimb_slip'] = (df['FrontLimb_y'] > df['Position_y']) & (df['TailBase_x'] < (df['End_x_adjust'] - 1))
df['FrontLimb_slip'] = df['FrontLimb_slip'].fillna(False)
df['HindLimb_slip'] = (df['HindLimb_y'] > df['Position_y']) & (df['TailBase_x'] < (df['End_x_adjust'] - 1))
df['HindLimb_slip'] = df['HindLimb_slip'].fillna(False) 

# Apply the function to create the 'Immobile' column
df['Immobile'] = df['Speed_cm/s'] < 0.05 * df['Speed_cm/s'].mean()

In [ ]:
# Create a custom function to calculate 'Time_to_descend'
def calculate_time_to_descend(group):
    threshold = 7
    if (group['TailBase_to_end'] <= threshold).any():
        # Find the index of the first observation where 'TailBase_to_end' is below the threshold
        index_below_threshold = (group['TailBase_to_end'] <= threshold).idxmax()
        # Count the number of rows before the first observation below the threshold
        rows_before_descend = index_below_threshold + 1
        # Calculate 'Time_to_descend' by dividing rows_before_descend by the frame rate
        return rows_before_descend / group['Framerate'].iloc[0]
    else:
        return np.nan

# Calculate 'Time_to_descend' for each 'Name File' group
result = df.groupby('Name File').apply(lambda x: calculate_time_to_descend(x))

# Merge the result back into the original DataFrame using 'Name File' as the key
df = df.merge(result.rename('Time_to_descend'), left_on='Name File', right_index=True)

#Get the time the mice spend in the end of the bar
df['Time_to_descend_end'] = df['Time_to_cross_s'] - df['Time_to_descend']

In [ ]:
import numpy as np

# Function to calculate time after a 7cm decrease
def calculate_time_after_7cm(group):
    initial_value = group['TailBase_to_end'].iat[0]  # Get the initial value for the group
    threshold = 7

    # Find the index of the first row where 'TailBase_to_end' decreases by 7 or more
    first_decrease_idx = (group['TailBase_to_end'] <= (initial_value - threshold)).idxmax()

    if not first_decrease_idx:
        return np.nan  # If no such decrease is found, return NaN

    # Calculate the time in seconds based on the index
    time_seconds = first_decrease_idx / group['Framerate'].iat[0]
    
    return time_seconds

# Apply the custom function to each 'Name File' group
time_after_7cm = df.groupby('Name File').apply(calculate_time_after_7cm).reset_index()
time_after_7cm.columns = ['Name File', 'Time_after_first_7cm']

# Merge the result back to the original DataFrame
df = df.merge(time_after_7cm, on='Name File', how='left')


##### Save the data frame here!

In [ ]:
save_path = r'C:\Users\UCPH\gabi\thesis analysis\BB_preprocess.csv'  # Provide your desired file name

# Save the DataFrame to a CSV file
df.to_csv(save_path, index=False)  # Set index=False to exclude the index column

#### Don't run this any further

In [ ]:
# Filter the DataFrame for rows where 'Dragging' is True and select specified columns
filtered_df = df[df['Failed_to_end']].copy()
columns_of_interest = ['Name File', 'TailBase_to_end','Failed_to_end', 'Time_to_descend', 'Time_to_cross_s']
filtered_df = filtered_df[columns_of_interest]

# Print the filtered DataFrame
filtered_df['Name File'].unique()

In [ ]:
filtered_df['Time_to_cross_s'].describe()

In [ ]:
## Generate a dragging collumn

# Ensure 'y_pixels' and 'Framerate' are integers
#df['y_pixels'] = df['y_pixels'].astype(int)
#df['Framerate'] = df['Framerate'].astype(int)

# Calculate the threshold for 'TailBase_y' based on 'y_pixels'
#df['threshold_y'] = np.where(df['y_pixels'] == 720, 10, (720 / 1080) * 10)

# Define a function to check the dragging condition
#def check_dragging_condition(group):
 #   return group['TailBase_y'].gt(group['Position_y'] + group['threshold_y']).all() and group['Immobile'].all()

# Apply the function to create the 'Dragging' column
#df['Dragging'] = False

# Check the dragging condition for each group of rows
#for _, group in df.groupby('Name File'):
 #   if group['Framerate'].iloc[0] == 15:
  #      if group.shape[0] >= 5 and not check_dragging_condition(group):
   #         df.loc[group.index, 'Dragging'] = True
    #elif group['Framerate'].iloc[0] == 60:
     #   if group.shape[0] >= 20 and not check_dragging_condition(group):
      #      df.loc[group.index, 'Dragging'] = True

In [ ]:
# Convert 'y_pixels' and 'Framerate' to integers
df['y_pixels'] = df['y_pixels'].astype(int)
df['Framerate'] = df['Framerate'].astype(int)
df['TailBase_y'] = df['TailBase_y'].astype(float)
df['Position_y'] = df['Position_y'].astype(float)

# Define a function to check the Dragging criteria
def check_dragging(row):
    if row['y_pixels'] == 720:
        threshold = 10
    elif row['y_pixels'] == 1080:
        threshold = (720 / 1080) * 10
    else:
        return None

    name_file_group = df.groupby('Name File').get_group(row['Name File'])
    if row['Framerate'] == 15:
        required_rows = 3
    elif row['Framerate'] == 60:
        required_rows = 12
    else:
        return None

    consecutive_count = 0  # Initialize a count for consecutive rows

    for _, group_row in name_file_group.iterrows():
        if (
            group_row['TailBase_y'] > (row['Position_y'] - threshold)
            and not group_row['Immobile']
        ):
            consecutive_count += 1
            if consecutive_count >= required_rows:
                return True
        else:
            consecutive_count = 0  # Reset the count if the criteria aren't met

    return False  # Return False if consecutive required frames are not found

# Apply the function to create the 'Dragging' column
df['Dragging'] = df.apply(check_dragging, axis=1)

In [ ]:
# Define a function to determine Framerate and y_pixels based on Cohort
def set_threshold(row):
    if row['y_pixels'] == 720:
         return 10
    elif row['y_pixels'] == 1080:
        return (720 / 1080) * 10
    else:
        return None

# Apply the function to create the 'Framerate' and 'y_pixels' columns
df['Threshold'] = df.apply(set_threshold, axis=1)

In [ ]:
# Filter the DataFrame for rows where 'Dragging' is True and select specified columns
filtered_df = df[df['Dragging']].copy()
columns_of_interest = ['Framerate', 'y_pixels', 'Name File', 'Position_y', 'TailBase_y', 'Threshold','Immobile', 'Dragging']
filtered_df = filtered_df[columns_of_interest]

# Print the filtered DataFrame
filtered_df.head()

In [ ]:
df[['TailBase_to_end', 'Start_y_adjust','End_y_adjust', 'Position_y']].head(50)

### Functions

In [3]:
def preprocess(df):

    """
    Takes the input file and returns a dataframe with the angles (both corrected and uncorrected)
    Parameters
    ----------

    df : pandas dataframe
        output .csv from the deeplabcut analysis.

    Returns
    -------
    df : pandas dataframe
        Same dataframe as input cleaner with generlized column names.
    """

    #seperate x, y and likelihood into 3 dfs for renaming and preprocessing
    df_x = df[df.columns[[1, 4, 7, 10, 13, 16]]]
    df_y = df[df.columns[[2, 5, 8, 11, 14, 17]]]
    df_like = df[df.columns[[3, 6, 9, 12, 15, 18]]]

    #drop the unneeded cols
    df_x = df_x.drop(0)
    df_y = df_y.drop(0)
    df_x = df_x.drop(2)
    df_y = df_y.drop(2)

    df_like = df_like.drop(0)
    df_like = df_like.drop(2)

    #Rename variables
    df_x.columns=df_x.iloc[0]
    df_x = df_x.drop(1)
    df_x.rename(columns = {'Nose':'Nose_x', 'Frontlimb':'FrontLimb_x', 'HindLimb':'HindLimb_x', 'TailBase':'TailBase_x', 'Start':'Start_x', 'End':'End_x'}, inplace = True)


    df_y.columns=df_y.iloc[0]
    df_y = df_y.drop(1)
    df_y.rename(columns = {'Nose':'Nose_y', 'Frontlimb':'FrontLimb_y', 'HindLimb':'HindLimb_y', 'TailBase':'TailBase_y', 'Start':'Start_y', 'End':'End_y'}, inplace = True)


    df_like.columns=df_like.iloc[0]
    df_like = df_like.drop(1)
    df_like.rename(columns = {'Nose':'Nose_likelihood', 'Frontlimb':'FrontLimb_likelihood', 'HindLimb':'HindLimb_likelihood', 'TailBase':'TailBase_likelihood', 'Start':'Start_likelihood', 'End':'End_likelihood'}, inplace = True)

    #merge the dataframes
    df_all = df_x.join(df_y)
    df_all = df_all.join(df_like)
    df_all = df_all.astype(float)

    return df_all

 

def files_from_folder(path):

    """
    Takes the input path and returns a dataframe with the concatanated data found in the folder

    Parameters
    ----------
    Path : input path for the folder where the files to be loaded are

    Returns
    ------
    df : pandas dataframe
        A concatanated dataframe with all the values
    """

    import glob
    import os.path
    import pandas as pd

    csv_files = glob.glob(os.path.join(path, "*.csv"))
    df = []

    # loop over the list of csv files
    for f in csv_files:

        # read the csv file
        file = pd.read_csv(f, header = None)
        file = preprocess(file)
        file['Name File'] = os.path.basename(f)
        file['Name File'] = file['Name File'].str.split('DLC_').str[0]
        df.append(file)
    df_con = pd.concat(df)

    return df_con